In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from math import sqrt


In [2]:

# Load the training and testing datasets
train_data = pd.read_csv('../../2nd-Comp-Data/train.csv')
test_data = pd.read_csv('../../2nd-Comp-Data/test.csv')
test_data.drop('row ID', axis=1, inplace=True)
testOriginal = pd.read_csv('../../2nd-Comp-Data/test.csv')

# Separate features and target variable in the training data
X = train_data.drop('price_doc', axis=1)
y = train_data['price_doc']


In [3]:

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:

# Define numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create transformers for numerical and categorical features
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Combine transformers into a preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [5]:

# Create a pipeline with preprocessor and Lasso Regression model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso(alpha=1.0, random_state=42))
])

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict the target variable on the validation set
y_pred = model.predict(X_val)

# Evaluate the model
rmse = sqrt(mean_squared_error(y_val, y_pred))
print(f'Root Mean Squared Error on validation set: {rmse}')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.1508150058008875e+19, tolerance: 6903319298803825.0
  model = cd_fast.sparse_enet_coordinate_descent(


Root Mean Squared Error on validation set: 13785013.675709365


In [6]:
test_predictions = model.predict(test_data)
result_df = pd.DataFrame({'row ID': testOriginal['row ID'], 'price_doc': test_predictions.flatten()})
result_df.to_csv('Day5.csv', index=False)

In [7]:
X_val.shape

(36302, 271)